![3.1](img/3.1.png)

![3.2](img/3.2.png)

### 3.3.1 A simple self-attention mechanism without trainable weights

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
# Calculate the itermediate attention scores between 
# the query token x2 and each input token 
# by calculating the dot product

query = inputs[1] # x2
attn_scores_2 = torch.empty(inputs.shape[0])
# attn_scores_3 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)
    # attn_scores_3[i] = torch.matmul(query, x_i.T)

print(attn_scores_2)
# print(attn_scores_3)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
# Normalize each of attention scores
# The goal is to obtain attention weights
# that sum up to 1

attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights: ", attn_weights_2_tmp)
print("Sum: ", attn_weights_2_tmp.sum())

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [5]:
# Normalize using the Softmax function

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights: ", attn_weights_2_naive)
print("Sum: ", attn_weights_2_naive.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [6]:
# PyTorch softmax

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights: ", attn_weights_2)
print("Sum: ", attn_weights_2.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [7]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Computing attention weights for all input tokens

In [8]:
# Calculating attention score with for loop
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
    
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
# Calculating attention score with matrix multiplication

attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
# Normalize each row 

attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [11]:
# verify that the rows sum to 1
print(attn_weights.sum(dim=-1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [12]:
# Compute context vector via matrix multiplication

all_context_vec = attn_weights @ inputs
print(all_context_vec) 

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [13]:
print("Previously 2nd context vector: ", context_vec_2)

Previously 2nd context vector:  tensor([0.4419, 0.6515, 0.5683])


## 3.4 Implementing self-attention with trainable weights

![3.13](img/3.13.png)

In [14]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [15]:
keys = inputs @ W_key
value = inputs @ W_value

print("keys.shape: ", keys.shape)
print("value.shape: ", value.shape)

keys.shape:  torch.Size([6, 2])
value.shape:  torch.Size([6, 2])


In [16]:
# compute attention score w22 
# q @ kT

keys2 = keys[1]
attn_scores_22 = query_2.dot(keys2) # torch.dot(query_2, keys2)
print(attn_scores_22)

attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor(1.8524)
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [17]:
# Normalize the attention scores
# softmax(q @ kT / sqrt(d_k))

d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [18]:
# calculate context vector
# qsoftmax(q @ kT / sqrt(d_k)) @ v

context_vec_2 = attn_weights_2 @ value
print(context_vec_2)

tensor([0.3061, 0.8210])


In [19]:
# Generalize to compute all context vectors

query = inputs @ W_query
key = inputs @ W_key
value = inputs @ W_value
d_k = key.shape[-1]

attn_scores = query @ key.T
attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
context = attn_weights @ value

print(context)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


### 3.4.2 Implementing a compact self-attention python class

In [53]:
import torch.nn as nn

class SelfAttentionv1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        query = x @ self.W_query
        key = x @ self.W_key
        value = x @ self.W_value

        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1] ** 0.5, dim=-1)
        context = attn_weights @ value

        return context

        

In [54]:
torch.manual_seed(123)
sa_v1 = SelfAttentionv1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [55]:
# Implement a self-attention class using PyTorch's Linear Layers

class SelfAttentionV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1] ** 0.5, dim=-1)

        context = attn_weights @ value
        return context

In [56]:
torch.manual_seed(123)
sa_v2 = SelfAttentionV2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


#### Exercise 3.1 Comparing SelfAttention_v1 and SelfAttention_v2

Your task is to correctly assign the weights from an instance of SelfAttention_v2
to an instance of SelfAttention_v1.

In [58]:
# print(sa_v1.W_query, sa_v1.W_query.shape)
# print(sa_v2.W_q.weight, sa_v2.W_q.weight.shape)

sa_v1.W_query = nn.Parameter(sa_v2.W_q.weight.T)
sa_v1.W_key = nn.Parameter(sa_v2.W_k.weight.T)
sa_v1.W_value = nn.Parameter(sa_v2.W_v.weight.T)

print(sa_v1(inputs))
print(sa_v2(inputs))


tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)
tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


## 3.5 Hiding future words with causal attention

![3.19](img/3.19.png)

In [77]:
# Implement a causal attention mask
# Attn_score (unnormalized) -> Attn_weights (normalized) -> Masked attn_scores (unnormalized) - Masked attn_weight (normalized)
    
class CausalAttentionV1(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        attn_scores = query @ key.T
        attn_weights = torch.softmax(attn_scores / key.shape[-1] ** 0.5, dim=-1)
        # print(attn_weights)

        context_len = attn_scores.shape[0]
        mask_simple = torch.tril(torch.ones(context_len, context_len))
        print(mask_simple)

        masked_attn_scores = attn_weights * mask_simple
        print(masked_attn_scores)

        row_sum = torch.sum(masked_attn_scores, dim=-1, keepdim=True)
        mask_simple_norm = masked_attn_scores / row_sum
        print(mask_simple_norm)

        context = attn_weights @ value
        return context

In [78]:
torch.manual_seed(123)
causal_attn = CausalAttentionV1(3, 6)
print(causal_attn(inputs))

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1654, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1771, 0.1558, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1770, 0.1559, 0.1565, 0.0000, 0.0000, 0.0000],
        [0.1739, 0.1594, 0.1598, 0.1689, 0.0000, 0.0000],
        [0.1710, 0.1606, 0.1610, 0.1689, 0.1745, 0.0000],
        [0.1759, 0.1582, 0.1586, 0.1690, 0.1747, 0.1637]],
       grad_fn=<MulBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5321, 0.4679, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3616, 0.3185, 0.3198, 0.0000, 0.0000, 0.0000],
        [0.2627, 0.2408, 0.2413, 0.2552, 0.0000, 0.0000],
        [0.2046, 0.1921, 0.1926, 0.2020, 0.2088, 0.0000],
        [0.1759, 0.1582, 0.1586, 0.1690, 0.1747, 0.1637]],
       grad_fn=<DivBackward0>)
tensor([[-0.5232, -0.2089,  0.2713,

In [ ]:
# Implement a more efficient masked attention
# Attn_score (unnormalized) -> Masked attn_scores (unnormalized) - Masked attn_weight (normalized)

class CausalAttentionV2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        attn_scores = query @ key.T

        context_len = attn_scores.shape[0]
        mask_simple = torch.triu(torch.ones(context_len, context_len), diagonal=1) == 1
        print(mask_simple)

        masked_attn_scores = torch.masked_fill(attn_scores, mask_simple.bool(), -torch.inf)
        print(masked_attn_scores)

        attn_weights = torch.softmax(masked_attn_scores / key.shape[-1] ** 0.5, dim=-1)
        print(attn_weights)

        context = attn_weights @ value
        return context
    
torch.manual_seed(123)
causal_attn = CausalAttentionV2(3, 6)
print(causal_attn(inputs))

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])
tensor([[ 0.0325,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.2287, -0.5432,    -inf,    -inf,    -inf,    -inf],
        [-0.2283, -0.5391, -0.5290,    -inf,    -inf,    -inf],
        [-0.1498, -0.3637, -0.3579, -0.2212,    -inf,    -inf],
        [-0.1579, -0.3127, -0.3064, -0.1891, -0.1082,    -inf],
        [-0.1599, -0.4197, -0.4130, -0.2573, -0.1763, -0.3359]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5321, 0.4679, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3616, 0.3185, 0.3198, 0.0000, 0.0000, 0.0000],
        [0.2627, 0.2408, 0.2413, 0.2552, 0.0000, 0.0000],
        [0.2046, 0.1921, 0.1926,

### 3.5.2 Masking additional attention weights with dropout

In [100]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(example)
out = dropout(example)
print(out)

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


### 3.5.3 Implementing a compact causal attention class

In [101]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [109]:
# Implement a CausalAttention with Dropout

class CausalAttentionV3(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()

        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        attn_scores = torch.matmul(query, key.transpose(1,2))

        # masked_attn_scores = torch.masked_fill(attn_scores, self.mask.bool(), -torch.inf)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) # perform in-place operations
        attn_weights = torch.softmax(attn_scores / key.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context = attn_weights @ value
        return context
    
torch.manual_seed(123)
context_length = batch.shape[1]
causal_attn = CausalAttentionV3(d_in, d_out, context_length, 0.0)
context_vecs = causal_attn(batch)
print(context_vecs.shape)

torch.Size([2, 6, 2])


## 3.6 Extending single-head attention to multi-head attention

In [114]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()

        self.heads = nn.ModuleList([
            CausalAttentionV3(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)
            ])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    

torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, 2)
context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])


#### Exercise 3.2 Returning two-dimensional embedding vectors

Change the input arguments for the MultiHeadAttentionWrapper(..., num_
heads=2) call such that the output context vectors are two-dimensional instead of
four dimensional while keeping the setting num_heads=2

In [115]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 1
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, 2)
context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]],

        [[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 2])


### 3.6.2 Implementing multi-head attention with weight splits

In [117]:
# Let's look at attn_scores = queries @ keys.transpose(2,3)


# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

out = a @ a.transpose(2, 3)
print(out)
print(out.shape)

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])
torch.Size([1, 2, 3, 3])


In [120]:
# The following is the equivalent way to compute the matrix multiplication
# for each head separately

first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print(first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print(second_res)

tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])
tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [140]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, n_heads, qkv_bias=False):
        super().__init__()

        self.w_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_v = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)

        self.n_heads = n_heads
        self.head_dims = d_out // n_heads
        self.projection = nn.Linear(d_out, d_out, bias=qkv_bias)

        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, context_length, d_in = x.shape

        query = self.w_q(x)
        key = self.w_k(x)
        value = self.w_v(x)

        query = query.view(b, context_length, self.n_heads, self.head_dims)
        key = key.view(b, context_length, self.n_heads, self.head_dims)
        value = value.view(b, context_length, self.n_heads, self.head_dims)

        query.transpose_(1, 2)
        key.transpose_(1, 2)
        value.transpose_(1, 2)

        print(query.shape)

        attn_score = query @ key.transpose(3, 2)

        attn_score.masked_fill_(self.mask.bool(), -torch.inf)

        attn_weights = torch.softmax(attn_score / key.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        print(attn_weights.shape)

        

        context = attn_weights @ value
        context.transpose_(2, 1)
        context = context.view(b, context_length, self.n_heads*self.head_dims)
        print(context.shape)

        context_proj = self.projection(context)
        print(context_proj.shape)

        return context_proj

        

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, n_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

torch.Size([2, 2, 6, 1])
torch.Size([2, 2, 6, 6])
torch.Size([2, 6, 2])
torch.Size([2, 6, 2])
tensor([[[ 0.1257, -0.1968],
         [ 0.1010, -0.2929],
         [ 0.0922, -0.3233],
         [ 0.0759, -0.2952],
         [ 0.0705, -0.2897],
         [ 0.0641, -0.2798]],

        [[ 0.1257, -0.1968],
         [ 0.1010, -0.2929],
         [ 0.0922, -0.3233],
         [ 0.0759, -0.2952],
         [ 0.0705, -0.2897],
         [ 0.0641, -0.2798]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 6, 2])
